In [2]:
pip install catboost

     |████████████████████████████████| 22.0 MB 557 kB/s eta 0:00:01     |█████████████████████           | 14.5 MB 380 kB/s eta 0:00:20
You should consider upgrading via the '/usr/local/Cellar/jupyterlab/2.2.8/libexec/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
from catboost import CatBoostRanker, Pool, MetricVisualizer
from copy import deepcopy
import numpy as np
import os
import pandas as pd

In [2]:
train_df = pd.read_csv("imat2009_new_split/imat2009_train_new.txt", header = None)
test_df = pd.read_csv("imat2009_new_split/imat2009_test_new.txt", header = None)

In [3]:
res = []
b = np.arange(1,256,1)

for i in range(train_df.shape[0]):
    feature = []
    value = []
    res_dict = dict.fromkeys(b)
    splitted = train_df[0][i].split()
    relevance = float(splitted[0])
    query_id = int(splitted[-1])
    for j in range(1, len(train_df[0][i].split())-2):
        feature.append(int(train_df[0][i].split()[j].split(':')[0]))
        value.append(float(train_df[0][i].split()[j].split(':')[1]))  
    for i in range(len(feature)):
        res_dict.update({'relevance': relevance,
                    'query_id': query_id, feature[i]:value[i]})
    res.append(res_dict)
train_df = pd.DataFrame.from_dict(res)


In [4]:
res = []
b = np.arange(1,256,1)
for i in range(test_df.shape[0]):
    feature = []
    value = []
    res_dict = dict.fromkeys(b)
    splitted = test_df[0][i].split()
    relevance = float(splitted[0])
    query_id = int(splitted[-1])
    for j in range(1, len(test_df[0][i].split())-2):
        feature.append(int(test_df[0][i].split()[j].split(':')[0]))
        value.append(float(test_df[0][i].split()[j].split(':')[1]))  
    for i in range(len(feature)):
        res_dict.update({'relevance': relevance,
                    'query_id': query_id, feature[i]:value[i]})
    res.append(res_dict)
test_df = pd.DataFrame.from_dict(res)


In [5]:
train_df_new = train_df.fillna(0)

In [6]:
test_df_new = test_df.fillna(0)

In [7]:
train_df_new.shape, test_df_new.shape

((77714, 257), (19576, 257))

In [8]:
test_df_new

,1,2,3,4,5,6,7,8,9,10,...,248,249,250,251,252,253,254,255,relevance,query_id
0,0.000000,0.0,0.030899,0.00,0.000000,0.0,0.703645,0.000000,0.000005,0.000000,...,0,0,0,0,0,0,0,0,1.0,855
1,0.000000,0.0,0.026270,0.00,0.000000,0.0,0.702928,0.611050,0.000021,0.001513,...,0,0,0,0,0,0,0,0,2.0,855
2,0.000000,1.0,0.047705,1.00,0.000000,0.0,0.653191,0.659319,0.000000,0.001146,...,0,0,0,0,0,0,0,0,2.0,855
3,0.000000,0.0,0.000000,0.00,0.000208,0.0,0.541343,0.576655,0.000003,0.000000,...,0,0,0,0,0,0,0,0,0.0,855
4,0.000074,0.0,0.026270,0.00,0.063105,0.0,0.243532,0.703421,0.042696,0.001513,...,0,0,0,0,0,0,0,0,1.0,855
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19571,0.000038,0.0,0.000000,0.00,0.000000,0.0,0.550233,0.000000,0.037738,0.000000,...,0,0,0,0,0,0,0,0,2.0,8488
19572,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.211102,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0.5,8488
19573,0.000000,0.0,0.044970,0.81,0.000000,0.0,0.591601,0.609226,0.000002,0.027377,...,0,0,0,0,0,0,0,0,2.0,8488
19574,0.000152,0.0,0.000000,1.00,0.000000,0.0,0.638857,0.619126,0.111746,0.003760,...,0,0,0,0,0,0,0,0,1.0,13354


In [9]:
X_train = train_df_new.drop(['relevance', 'query_id'], axis=1).values
y_train = train_df_new['relevance'].values
queries_train = train_df_new['query_id'].values

X_test = test_df_new.drop(['relevance', 'query_id'], axis=1).values
y_test = test_df_new['relevance'].values
queries_test = test_df_new['query_id'].values


In [10]:
X_train.shape, X_test.shape

((77714, 255), (19576, 255))

In [11]:
num_documents = X_train.shape[0]
print(num_documents)

77714


In [12]:
X_train.shape[1]

255

In [13]:
from collections import Counter
Counter(y_train).items()

dict_items([(1.0, 20086), (0.0, 25776), (2.0, 24424), (4.0, 952), (3.0, 1744), (0.5, 1982), (1.5, 1033), (0.25, 77), (1.33333, 110), (1.2, 3), (2.37037, 39), (0.666671, 340), (2.33333, 79), (0.333329, 268), (2.16049, 19), (2.5, 337), (2.87037, 26), (1.66667, 107), (2.12037, 4), (2.25, 19), (2.24074, 25), (0.2, 10), (1.6, 6), (0.8, 5), (0.6, 10), (0.875, 1), (2.66667, 31), (3.1625, 2), (1.75, 12), (0.75, 55), (2.61111, 4), (0.222229, 1), (0.4, 5), (1.25, 23), (1.97143, 2), (3.5, 16), (2.24691, 10), (2.16667, 1), (1.95239, 1), (1.4, 4), (3.66667, 5), (3.8, 2), (0.125, 1), (2.05556, 2), (3.33333, 4), (2.2, 5), (2.58025, 2), (1.16667, 2), (2.91358, 1), (2.07407, 3), (2.11729, 1), (3.25, 1), (2.375, 1), (3.21666, 1), (2.74074, 5), (2.12346, 3), (0.166671, 8), (0.833329, 5), (1.14286, 1), (3.53, 1), (3.4, 1), (2.75, 1), (3.58125, 1), (2.40741, 1), (0.583329, 1), (1.8, 1), (2.42857, 1), (2.0463, 1), (1.77143, 1), (3.75, 1), (0.888886, 1)])

In [14]:
max_relevance = np.max(y_train)
print(max_relevance)
y_train /= max_relevance
y_test /= max_relevance

4.0


In [15]:
num_queries = np.unique(queries_train).shape[0]
print(num_queries)

7300


In [16]:
train = Pool(
    data=X_train,
    label=y_train,
    group_id=queries_train
)

test = Pool(
    data=X_test,
    label=y_test,
    group_id=queries_test
)

In [17]:
default_parameters = {
    'iterations': 2000,
    'custom_metric': ['NDCG', 'PFound', 'AverageGain:top=10'],
    'verbose': False,
    'random_seed': 0,
}

parameters = {}

In [18]:
def fit_model(loss_function, additional_params=None, train_pool=train, test_pool=test):
    parameters = deepcopy(default_parameters)
    parameters['loss_function'] = loss_function
    parameters['train_dir'] = loss_function
    
    if additional_params is not None:
        parameters.update(additional_params)
        
    model = CatBoostRanker(**parameters)
    model.fit(train_pool, eval_set=test_pool, plot=True)
    
    return model

In [19]:
%%time

fit_model('PairLogit')

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

CPU times: user 16min 9s, sys: 17.7 s, total: 16min 27s
Wall time: 10min


In [20]:
%%time

fit_model('PairLogitPairwise')

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

CPU times: user 3h 38min 45s, sys: 2min 10s, total: 3h 40min 55s
Wall time: 1h 35min 39s


In [21]:
%%time

fit_model('YetiRank')

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

CPU times: user 11min 40s, sys: 8.9 s, total: 11min 49s
Wall time: 4min 51s


In [23]:
%%time

fit_model('YetiRankPairwise')

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

CPU times: user 2h 16s, sys: 29.4 s, total: 2h 45s
Wall time: 36min 48s


In [24]:
widget = MetricVisualizer(['PairLogit', 'PairLogitPairwise', 'YetiRank', 'YetiRankPairwise'])
widget.start()

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))